# Download observational data from Frost

[Frost](https://frost.met.no/index.html) is an API which gives access to MET Norway's archive of historical weather and climate data. 

## Get access
- to access the API you need to [create a user](https://frost.met.no/auth/requestCredentials.html)

## How to use Frost
- [basic introduction](https://frost.met.no/howto.html) to help you learn to use Frost
- [Examples](https://frost.met.no/examples2.html) of how to use Frost

## How to find the variable?
- [Browse weather elements](https://frost.met.no/elementtable)

The following script is based on the [example](https://frost.met.no/python_example.html) provided by Frost documentation

New client credentials have been successfully created for email address franziska.hellmuth@geo.uio.no. Your client ID is:

In [1]:
client_id = 'd2e8db6e-9f6b-4cff-a337-3accf09bc8d8'

In [2]:
# import packages
import requests
import pandas as pd
import xarray as xr
import numpy as np

To find station information use the Norwegian Centre for Climate services: <https://seklima.met.no/stations/>

## We will use Andøya airport

- Municipality: Andøy
- County: Nordland
- Station number (id): SN87110
- Height above mean sea level: 10 m
- Latitude: 69.3073º N
- Longitude: 16.1312º E
- Operating period: 01.01.1958 - now
- WMO number: 1010
- WIGOS number: 0-20000-0-01010
- Station holder: Met.no, Avinor

In [3]:
station = 'SN87110'  # based on the information taken from seklima

Define the variables to be downloaded after you [browsed weather elements](https://frost.met.no/elementtable)


In [ ]:
_xx = xr.open_dataset('/scratch/franzihe/output/Met-No_obs/SN87110/air_pressure_at_sea_level_202104.nc')

In [16]:
elements = [
    'air_temperature', 
    'wind_speed', 
    'wind_from_direction', 
    'air_pressure_at_sea_level', 
    'sum(precipitation_amount PT1H)',
    # 'sum(precipitation_amount P1D)',      # error when downloading
    # 'sum(precipitation_amount PT12H)',    # error when downloading
    'cloud_area_fraction',
    'cloud_area_fraction1',
    'cloud_area_fraction2',
    'cloud_area_fraction3',
    'cloud_base_height1',
    'cloud_base_height2',
    'cloud_base_height3',

    ]

In [17]:
reference_time = [
    '2021-03-01/2021-03-31', 
    '2021-04-01/2021-04-30'
    ]   # start and end of data which shall be retrieved

In [18]:
for ref_time in reference_time:

    for var in elements:
        
        # retrieve data from Frost using the requests.get function.
        # Define endpoint and parameters
        endpoint = 'https://frost.met.no/observations/v0.jsonld'
        parameters = {
            'sources': station, 
            'elements': var,
            'referencetime': ref_time,
        }
        # Issue an HTTP GET request
        r = requests.get(endpoint, parameters, auth=(client_id,''))
        # Extract JSON data
        json = r.json()

        # Check if the request worked, print out any errors
        if r.status_code == 200:
            data = json['data']
            print('{} retrieved from frost.met.no!'.format(var))
        else:
            print('Error! Returned status code %s' % r.status_code)
            print('Message: %s' % json['error']['message'])
            print('Reason: %s' % json['error']['reason'])

        # This will return a Dataframe with all of the observations in a table format
        df = pd.DataFrame()
        for i in range(len(data)):
            row = pd.DataFrame(data[i]['observations'])
            row['referenceTime'] = data[i]['referenceTime']
            row['sourceId'] = data[i]['sourceId']
            df = df.append(row)

        df = df.reset_index()

        
# make a shorter and more readable table, you can use the code below.
        # These additional columns will be kept
        columns = ['sourceId','referenceTime','value','unit','timeOffset', 'timeResolution', 'level']
        try: 
            df = df[columns].copy()
        except KeyError:
            columns = ['sourceId','referenceTime','value','unit','timeOffset', 'timeResolution',]
            df = df[columns].copy()

        # Convert the time value to something Python understands
        df['referenceTime'] = pd.to_datetime(df['referenceTime'])
        
        if var == 'air_pressure_at_sea_level' or var == 'sum(precipitation_amount PT1H)':
            print('hourly data retrieved')
            df.drop(df[df['timeResolution'] != 'PT1H'].index, inplace=True)
            df.drop(df[df['timeOffset'] != 'PT0H'].index, inplace = True)
        elif var == 'cloud_area_fraction2' or var == 'cloud_area_fraction3':
            df.drop(df[df['timeResolution'] != 'PT30M'].index, inplace=True)
            df.drop(df[df['timeOffset'] != 'PT20M'].index, inplace = True)
        else:
            # select only 10-minute time resolution and timeOffset to be at 0H
            df.drop(df[df['timeResolution'] != 'PT10M'].index, inplace=True)
            df.drop(df[df['timeOffset'] != 'PT0H'].index, inplace = True)
        # reset the index to start at zero
        df.reset_index(drop=True, inplace = True)
        # rename the columns (useful for later when we create the xarray)
        df.rename(columns={'value':var, }, inplace=True)

        

        # create xarray DataArray and assign units
        try:
            dsx = df.to_xarray().drop_vars(['unit', 'timeOffset', 'timeResolution', 'level', 'sourceId'])
        except ValueError:
            dsx = df.to_xarray().drop_vars(['unit', 'timeOffset', 'timeResolution', 'sourceId'])
            
        attrs = {'units': ''}
        dsx['referenceTime'].assign_attrs(attrs)

        dsx['index'] = pd.to_datetime(dsx['referenceTime'].values)
        
        # rename index
        dsx = dsx.rename({'index': 'time'})
        # remove variable referenceTime
        dsx = dsx.drop('referenceTime')
        
        dsx['time']  = pd.DatetimeIndex(dsx['time'].values)
        
        # assign attributes to variable
        try:
            dsx[var] = dsx[var].assign_attrs({'units': df['unit'][0], df['level'][0]['levelType'] : str(df['level'][0]['value']) + df['level'][0]['unit']})
        except KeyError:
            dsx[var] = dsx[var].assign_attrs({'units': df['unit'][0], })
        # assign attributes to dataset
        dsx = dsx.assign_attrs({
            'Municipality': 'Andøy', 
            'County': 'Nordland',
            'Height above mean sea level': '10 m',
            'Station number (id)': 'SN87110',
            'Latitude': 69.3073,
            'Longitude': 16.1312,
            'WMO number': 1010})

        save_file = '/scratch/franzihe/output/Met-No_obs/{}/{}_{}{}.nc'.format(station, var, ref_time.split('-')[0], ref_time.split('-')[1])
        dsx.to_netcdf(path = save_file)

        print('File saved: {}'.format(save_file))



air_temperature retrieved from frost.met.no!
File saved: /scratch/franzihe/output/Met-No_obs/SN87110/air_temperature_202103.nc
wind_speed retrieved from frost.met.no!
File saved: /scratch/franzihe/output/Met-No_obs/SN87110/wind_speed_202103.nc
wind_from_direction retrieved from frost.met.no!
File saved: /scratch/franzihe/output/Met-No_obs/SN87110/wind_from_direction_202103.nc
air_pressure_at_sea_level retrieved from frost.met.no!
hourly data retrieved
File saved: /scratch/franzihe/output/Met-No_obs/SN87110/air_pressure_at_sea_level_202103.nc
sum(precipitation_amount PT1H) retrieved from frost.met.no!
hourly data retrieved
File saved: /scratch/franzihe/output/Met-No_obs/SN87110/sum(precipitation_amount PT1H)_202103.nc
cloud_area_fraction retrieved from frost.met.no!
File saved: /scratch/franzihe/output/Met-No_obs/SN87110/cloud_area_fraction_202103.nc
cloud_area_fraction1 retrieved from frost.met.no!
File saved: /scratch/franzihe/output/Met-No_obs/SN87110/cloud_area_fraction1_202103.nc
c